# Pandas DataFrame Parser

Pandas DataFrame은 Python 프로그래밍 언어에서 널리 사용되는 데이터 구조로, 데이터 조작 및 분석을 위해 흔히 사용됩니다. 구조화된 데이터를 다루기 위한 포괄적인 도구 세트를 제공하여, 데이터 정제, 변환 및 분석과 같은 작업에 다양하게 활용될 수 있습니다.

이 출력 파서는 사용자가 임의의 Pandas DataFrame을 지정하고 해당 DataFrame에서 데이터를 추출하여 형식화된 사전 형태로 데이터를 조회할 수 있는 LLM을 요청할 수 있게 해줍니다. 대규모 언어 모델은 누수 추상화라는 점을 기억하세요! 정의된 형식 지침에 따라 잘 형성된 쿼리를 생성하기 위해서는 충분한 용량을 가진 LLM을 사용해야 합니다.

쿼리를 수행하고자 하는 DataFrame에 대해 Pandas의 DataFrame 객체를 사용하여 선언하세요.


In [1]:
import pprint
from typing import Any, Dict

import pandas as pd
from langchain.output_parsers import (
    PandasDataFrameOutputParser,
)  # Pandas 데이터프레임 출력 파서
from langchain.prompts import PromptTemplate  # 프롬프트 템플릿
from langchain_openai import ChatOpenAI  # ChatOpenAI

In [2]:
# ChatOpenAI 모델 초기화
model = ChatOpenAI(temperature=0)

- `format_parser_output` 함수는 파서 출력을 사전 형식으로 변환하고 이를 예쁘게 출력합니다.
- 파서 출력(`parser_output`)은 `Dict[str, Any]` 타입의 매개변수로 받습니다.
- 출력된 각 키에 대해, 해당 키의 값을 `.to_dict()` 메소드를 사용하여 사전 형식으로 변환합니다.
- `pprint.PrettyPrinter` 클래스를 사용하여 변환된 출력을 가독성 높게 출력합니다. 출력 설정으로는 너비(`width`)를 4로, `compact` 모드를 `True`로 설정합니다.


In [3]:
# 문서화 목적으로만 사용됩니다.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():  # 파서 출력의 키들을 순회합니다.
        # 각 키의 값을 딕셔너리로 변환합니다.
        parser_output[key] = parser_output[key].to_dict()
    # 예쁘게 출력합니다.
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

- `titanic.csv` 데이터를 읽어온 뒤 DataFrame 을 로드하여 `df` 변수에 할당합니다.
- PandasDataFrameOutputParser를 사용하여 DataFrame을 파싱합니다.


In [6]:
# 원하는 Pandas DataFrame을 정의합니다.
df = pd.read_csv("./data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = PandasDataFrameOutputParser(dataframe=df)

- 컬럼 작업을 수행하는 예제입니다.
- 사용자 쿼리를 처리하기 위한 프롬프트를 설정합니다.
- `PromptTemplate`을 사용하여 쿼리 포맷과 관련 지시사항을 포함한 템플릿을 생성합니다.
- `parser.get_format_instructions()` 함수를 호출하여 포맷 지시사항을 가져옵니다.
- 프롬프트, 모델, 파서를 연결하여 처리 체인을 구성합니다.
- `chain.invoke` 메소드를 사용하여 `df_query`에 대한 처리를 실행합니다.
- `format_parser_output` 함수를 호출하여 파서의 출력을 포맷합니다.


In [8]:
# 열 작업 예시입니다.
df_query = "Retrieve the passenges ages."

# 프롬프트 설정
prompt = PromptTemplate(
    # 사용자 쿼리에 답합니다.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

chain = prompt | model | parser  # 체인 생성
parser_output = chain.invoke({"query": df_query})  # 체인 실행

format_parser_output(parser_output)  # 파서 출력 형식 지정

{'Age': {0: 22.0,
         1: 38.0,
         2: 26.0,
         3: 35.0,
         4: 35.0,
         5: nan,
         6: 54.0,
         7: 2.0,
         8: 27.0,
         9: 14.0,
         10: 4.0,
         11: 58.0,
         12: 20.0,
         13: 39.0,
         14: 14.0,
         15: 55.0,
         16: 2.0,
         17: nan,
         18: 31.0,
         19: nan}}


- 데이터 프레임의 첫 번째 행을 검색하는 쿼리를 처리합니다.
- `PromptTemplate`을 사용하여 사용자 쿼리에 대한 프롬프트를 설정합니다. 이때, 쿼리와 포맷 지시사항을 변수로 사용합니다.
- `parser.get_format_instructions()` 함수를 호출하여 포맷 지시사항을 가져옵니다.
- 프롬프트, 모델, 파서를 연결하여 처리 체인을 구성합니다.
- 처리 체인을 통해 쿼리를 실행하고, `chain.invoke` 함수를 사용하여 쿼리 결과를 처리합니다.
- `format_parser_output` 함수를 사용하여 파서의 출력 결과를 포맷합니다.


In [9]:
# 행 연산 예시입니다.
df_query = "Retrieve the first row."

# 프롬프트 설정
prompt = PromptTemplate(
    # 사용자 질의에 답합니다.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수
)

chain = prompt | model | parser  # 체인 생성
parser_output = chain.invoke({"query": df_query})  # 체인 실행

format_parser_output(parser_output)  # 파서 출력 형식 지정

{'0': {'Age': 22.0,
       'Cabin': nan,
       'Embarked': 'S',
       'Fare': 7.25,
       'Name': 'Braund, '
               'Mr. '
               'Owen '
               'Harris',
       'Parch': 0,
       'PassengerId': 1,
       'Pclass': 3,
       'Sex': 'male',
       'SibSp': 1,
       'Survived': 0,
       'Ticket': 'A/5 '
                 '21171'}}


- Pandas DataFrame의 특정 열에서 일부 행의 평균을 검색하는 작업 예제입니다.
- 사용자 쿼리에 대한 프롬프트를 설정합니다.
- `PromptTemplate`을 사용하여 쿼리 포맷과 관련 지시사항을 포함한 프롬프트를 구성합니다.
- `parser.get_format_instructions()`을 호출하여 포맷 지시사항을 가져옵니다.
- 프롬프트, 모델, 파서를 연결하여 처리 체인을 구성합니다.
- 처리 체인을 통해 사용자 쿼리(`df_query`)에 대한 파싱 작업을 수행합니다.
- 파싱된 출력 결과를 출력합니다.


In [19]:
# row 0 ~ 4의 평균 나이를 구합니다.
df["Age"].head().mean()

31.2

In [18]:
# 임의의 Pandas DataFrame 작업 예시, 행의 수를 제한합니다.
df_query = "Retrieve the average of the ages from row 0 to 4."

# 프롬프트 설정
prompt = PromptTemplate(
    # 사용자 쿼리에 답합니다.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수 설정
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },  # 부분 변수 설정
)

chain = prompt | model | parser  # 체인 생성
parser_output = chain.invoke({"query": df_query})  # 체인 실행

print(parser_output)  # 결과 출력

{'mean': 31.2}


- 사용자 쿼리에 대한 답변을 생성하기 위해 `PromptTemplate`을 설정합니다.
- `PromptTemplate`에서는 쿼리와 포맷 지시사항을 포함한 템플릿을 정의합니다.
- 포맷 지시사항은 `parser.get_format_instructions()`을 호출하여 얻습니다.
- `prompt`, `model`, `parser`를 연결하여 처리 체인을 구성합니다.
- 처리 체인을 통해 주어진 쿼리(`df_query`)에 대한 파싱 결과를 생성합니다.


In [27]:
# 잘못 형식화된 쿼리의 예시입니다.
df_query = "Retrieve the average Fare."

# 프롬프트 설정합니다.
prompt = PromptTemplate(
    # 사용자 쿼리에 답합니다.
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],  # 입력 변수를 지정합니다.
    # 부분 변수를 설정합니다.
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser  # 체인을 구성합니다.
parser_output = chain.invoke({"query": df_query})  # 체인을 호출하여 결과를 얻습니다.

In [28]:
# 결과 출력출력
parser_output

{'mean': 22.19937}

In [29]:
# 결과 검증
df["Fare"].mean()

22.19937